In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("washing_machine.csv")
data

,index,name,price,number_of_comment,brand,size,type,weight(kg),comment
0,0,Máy giặt Toshiba Inverter 8.5 Kg TW-BH95S2V WK,5999000,131,Toshiba,RxSxC 595 x 53 x 85 cm,Cửa trước,68,Ổn trong tầm giá Với giá này thì tôi hoàn toàn...
1,0,Máy giặt Toshiba Inverter 8.5 Kg TW-BH95S2V WK,5999000,131,Toshiba,RxSxC 595 x 53 x 85 cm,Cửa trước,68,Giao hàng nhanh có thợ lắp ráp ngay không phải...
2,0,Máy giặt Toshiba Inverter 8.5 Kg TW-BH95S2V WK,5999000,131,Toshiba,RxSxC 595 x 53 x 85 cm,Cửa trước,68,Mình nhận xét sau khi mua sản phẩm Máy giặt To...
3,0,Máy giặt Toshiba Inverter 8.5 Kg TW-BH95S2V WK,5999000,131,Toshiba,RxSxC 595 x 53 x 85 cm,Cửa trước,68,Dịch vụ rất tốt giao nhận nhiệt tình vui vẻ hỗ...
4,0,Máy giặt Toshiba Inverter 8.5 Kg TW-BH95S2V WK,5999000,131,Toshiba,RxSxC 595 x 53 x 85 cm,Cửa trước,68,Ai nói máy này ồn thì là nói xạo nhé Để chế độ...
...,...,...,...,...,...,...,...,...,...
2495,554,Máy giặt Aqua 11kg AQW-FW110FT-N - HÀNG CHÍNH ...,5959000,0,Aqua,NaN,Cửa trên,48,NaN
2496,555,Máy giặt Aqua Inverter 10.5 kg AQD-D1050E N,9290000,0,Aqua,595 x 595 x 845 cm,Cửa trước,80,NaN
2497,556,Máy Giặt Cửa Trước Samsung Inverter WW90J54E0B...,8290000,0,Samsung,NaN,NaN,NaN,NaN
2498,557,Máy giặt Electrolux Inverter 9 kg EWF9024BDWA ...,9490000,0,Electrolux,NaN,Cửa trước,NaN,NaN


In [5]:
data.dropna(subset=['comment'], inplace=True)

In [6]:
review_predict = data['comment']

In [7]:
sentiment = pd.read_csv('text.csv',index_col=[0])
sentiment

,text_lb,text
2,1.0,"Ấn tượng, tôi sẽ quay lại vào dịp nghỉ lần sau"
3,1.0,"Quá tuyệt vời , view đẹp , chủ nhà thân thiện ..."
6,1.0,"Phòng nghỉ đáng giá, tiện nghi cơ bản đầy đủ ^..."
7,1.0,Thuận tiện cho việc đi thăm thú cảnh quan.\r\n...
10,1.0,Nhân viên thân thiện
...,...,...
6994,1.0,"Anh chủ rất là nhiệt tình và vui vẻ, phòng sạc..."
6995,1.0,"Vị trí đẹp, mặc dù chúng tôi là khách duy nhất..."
6996,-1.0,"Không có chỗ đỗ xe, phải gửi xe ở ngoài rất tố..."
6997,1.0,"Gửi quản lý cấp cao của khách sạn, Rất tiếc hì..."


In [8]:
with open('vietnamese_stopwords.txt', encoding="utf8") as f:
    stopwords = []
    for line in f:
        stopwords.append("_".join(line.strip().split()))

In [9]:
import re
from pyvi import ViTokenizer
def preprocessor(text):
    corpus = []
    for i in range(0, len(text)):
        review = re.sub(r"http\S+", "", str(text[i]))
        review = re.sub(r"#\S+", "", review)
        review = re.sub(r"@\S+", "", review)
        review = re.sub('[_]',' ',review)
        review = re.sub('[^a-zA-Z_áàạảãăắằặẵẳâấầẩậẫđíỉìịĩóòỏọõôốồổộỗơớờởợỡéèẹẽẻêếềểệễúùủũụưứừửựữýỳỷỹỵÁÀẢÃẠĂẮẰẲẲẶẴÂẤẦẬẪẨĐÍÌỈỊĨÓÒỎỌÕÔỐỒỔỘỖƠỚỜỞỢỠÉÈẺẸẼÊẾỀỆỂỄÚÙỦŨỤƯỨỪỬỰỮÝỲỶỴỸ]',
                        ' ',review)
        review = ViTokenizer.tokenize(review)
        review = review.lower()
        review = review.split()
        review = [word for word in review if not word in  set(stopwords)]
        review = ' '.join(review)
        corpus.append(review)
    return corpus

In [10]:
X = sentiment['text'].values
corpus = preprocessor(X)

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer(analyzer='word', max_features=30000)
tfidf_vect.fit(corpus) 
X_data_tfidf =  tfidf_vect.transform(corpus)

from sklearn.model_selection import train_test_split
X = X_data_tfidf
y = sentiment['text_lb'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=102)

from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0)
clf.fit(X_train, y_train)

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
predictions = clf.predict(X_test)
print('accuracy:',accuracy_score(y_test,predictions))

accuracy: 0.8759615384615385


In [12]:
from sklearn import svm
classifier = svm.SVC(probability=True)
classifier.fit(X_train, y_train)
train_predictions = classifier.predict(X_train)
predictions = classifier.predict(X_test)
print('accuracy:',accuracy_score(y_test,predictions))
print('confusion matrix:\n',confusion_matrix(y_test,predictions))
print('classification report:\n',classification_report(y_test,predictions))

accuracy: 0.885576923076923
confusion matrix:
 [[ 84 100]
 [ 19 837]]
classification report:
               precision    recall  f1-score   support

        -1.0       0.82      0.46      0.59       184
         1.0       0.89      0.98      0.93       856

    accuracy                           0.89      1040
   macro avg       0.85      0.72      0.76      1040
weighted avg       0.88      0.89      0.87      1040



In [13]:
X_1 = review_predict.values
corpus1 = preprocessor(X_1)
X_data_tfidf_1 =  tfidf_vect.transform(corpus1)

In [14]:
proba = classifier.predict_proba(X_data_tfidf_1)

In [15]:
df1 = pd.DataFrame(data=proba, columns=['Negative', 'Positive'])

In [16]:
data = pd.concat([data, df1], axis = 1).drop(columns = ['comment'])
data

,index,name,price,number_of_comment,brand,size,type,weight(kg),Negative,Positive
0,0.0,Máy giặt Toshiba Inverter 8.5 Kg TW-BH95S2V WK,5999000.0,131.0,Toshiba,RxSxC 595 x 53 x 85 cm,Cửa trước,68,0.102036,0.897964
1,0.0,Máy giặt Toshiba Inverter 8.5 Kg TW-BH95S2V WK,5999000.0,131.0,Toshiba,RxSxC 595 x 53 x 85 cm,Cửa trước,68,0.284347,0.715653
2,0.0,Máy giặt Toshiba Inverter 8.5 Kg TW-BH95S2V WK,5999000.0,131.0,Toshiba,RxSxC 595 x 53 x 85 cm,Cửa trước,68,0.791268,0.208732
3,0.0,Máy giặt Toshiba Inverter 8.5 Kg TW-BH95S2V WK,5999000.0,131.0,Toshiba,RxSxC 595 x 53 x 85 cm,Cửa trước,68,0.252180,0.747820
4,0.0,Máy giặt Toshiba Inverter 8.5 Kg TW-BH95S2V WK,5999000.0,131.0,Toshiba,RxSxC 595 x 53 x 85 cm,Cửa trước,68,0.287539,0.712461
...,...,...,...,...,...,...,...,...,...,...
2473,534.0,Máy Giặt Cửa Ngang Inverter Electrolux EWF1402...,14690000.0,2.0,Electrolux,Cao 850 x Rông 600 x Sâu 669 mm,Cửa trước,70,NaN,NaN
2474,534.0,Máy Giặt Cửa Ngang Inverter Electrolux EWF1402...,14690000.0,2.0,Electrolux,Cao 850 x Rông 600 x Sâu 669 mm,Cửa trước,70,NaN,NaN
2476,536.0,Máy giặt Electrolux Inverter 9 kg EWF12938,10900000.0,2.0,Electrolux,60 x 669 x 85 cm D x R x C,Cửa trước,NaN,NaN,NaN
2477,536.0,Máy giặt Electrolux Inverter 9 kg EWF12938,10900000.0,2.0,Electrolux,60 x 669 x 85 cm D x R x C,Cửa trước,NaN,NaN,NaN


In [17]:
group_data = data.groupby(['index', 'name', 'price', 'number_of_comment', 'size', 'brand', 'type', 'weight(kg)'], as_index=False, dropna=False).mean()
#data.groupby(['index', 'name', 'price', 'number_of_comment'], as_index=False).mean()
group_data

,index,name,price,number_of_comment,size,brand,type,weight(kg),Negative,Positive
0,0.0,Máy giặt Toshiba Inverter 8.5 Kg TW-BH95S2V WK,5999000.0,131.0,RxSxC 595 x 53 x 85 cm,Toshiba,Cửa trước,68,0.214451,0.785549
1,1.0,Máy giặt Electrolux Inverter 8 Kg EWF8025DGWA,6589000.0,87.0,Cao 85cm Ngang 600cm Sâu 568cm,Electrolux,Cửa trước,NaN,0.231496,0.768504
2,2.0,Máy giặt Electrolux Inverter 8 Kg EWF8025DGWA,6590000.0,46.0,Cao 85cm Ngang 600cm Sâu 568cm,Electrolux,Cửa trước,NaN,0.287542,0.712458
3,3.0,Máy Giặt Sấy LG Inverter 9 Kg FV1409G4V - Chỉ ...,12479000.0,19.0,60 x 55 x 85 cm,LG,Cửa trước,69,0.182412,0.817588
4,4.0,Máy giặt Toshiba Inverter 8.5 Kg TW-BH95S2V WK,6049000.0,60.0,RxSxC 595 x 53 x 85 cm,Toshiba,Cửa trước,68,0.238306,0.761694
...,...,...,...,...,...,...,...,...,...,...
262,533.0,Máy Giặt Cửa Trước Samsung Inverter Addwash WW...,9990000.0,1.0,NaN,Samsung,NaN,NaN,NaN,NaN
263,534.0,Máy Giặt Cửa Ngang Inverter Electrolux EWF1402...,14690000.0,2.0,Cao 850 x Rông 600 x Sâu 669 mm,Electrolux,Cửa trước,70,NaN,NaN
264,536.0,Máy giặt Electrolux Inverter 9 kg EWF12938,10900000.0,2.0,60 x 669 x 85 cm D x R x C,Electrolux,Cửa trước,NaN,NaN,NaN
265,543.0,Máy giặt Electrolux Inverter 9.5 kg EWF9523BDWA,11190000.0,1.0,60 x 659 x 85 cm,Electrolux,Cửa trước,71,NaN,NaN


In [20]:
rank_data = group_data.drop(columns=['index', 'name', 'brand', 'size', 'type'])
rank_data.values

array([[5999000.0, 131.0, '68', 0.21445116036462045, 0.7855488396353812],
       [6589000.0, 87.0, nan, 0.23149577558165974, 0.7685042244183412],
       [6590000.0, 46.0, nan, 0.2875417441442242, 0.7124582558557755],
       ...,
       [10900000.0, 2.0, nan, nan, nan],
       [11190000.0, 1.0, '71', nan, nan],
       [nan, nan, nan, 0.27265343204939224, 0.7273465679506081]],
      dtype=object)

In [21]:
pd.set_option('display.max_rows', rank_data.shape[0]+1)
rank_data[lambda col :rank_data.columns] = rank_data[lambda col :rank_data.columns].replace(r'\D+', np.NaN, regex=True)
rank_data

,price,number_of_comment,weight(kg),Negative,Positive
0,5999000.0,131.0,68,0.214451,0.785549
1,6589000.0,87.0,NaN,0.231496,0.768504
2,6590000.0,46.0,NaN,0.287542,0.712458
3,12479000.0,19.0,69,0.182412,0.817588
4,6049000.0,60.0,68,0.238306,0.761694
5,6179000.0,16.0,63,0.132536,0.867464
6,6999000.0,37.0,63,0.174095,0.825905
7,3649000.0,48.0,NaN,0.330885,0.669115
8,7790000.0,32.0,68,0.245729,0.754271
9,7690000.0,16.0,68,0.253421,0.746579


In [22]:
from sklearn.impute import KNNImputer
data_imputed = rank_data.values
ix = [i for i in range(data_imputed.shape[1]) if i != 6]
X = data_imputed[:, ix]
imputer = KNNImputer()
imputer.fit(X)
Xtrans = imputer.transform(X)
X.shape

(267, 5)

In [23]:
rank_data = pd.DataFrame(Xtrans,index=rank_data.index, columns=rank_data.columns)

In [24]:
rank_data

,price,number_of_comment,weight(kg),Negative,Positive
0,5999000.0,131.0,68.0,0.214451,0.785549
1,6589000.0,87.0,53.6,0.231496,0.768504
2,6590000.0,46.0,53.6,0.287542,0.712458
3,12479000.0,19.0,69.0,0.182412,0.817588
4,6049000.0,60.0,68.0,0.238306,0.761694
5,6179000.0,16.0,63.0,0.132536,0.867464
6,6999000.0,37.0,63.0,0.174095,0.825905
7,3649000.0,48.0,29.2,0.330885,0.669115
8,7790000.0,32.0,68.0,0.245729,0.754271
9,7690000.0,16.0,68.0,0.253421,0.746579


In [25]:
from sklearn.preprocessing import StandardScaler

Standard_scaler = StandardScaler()
Xtrans = Standard_scaler.fit_transform(Xtrans)

In [26]:
rank_data_scaled = pd.DataFrame(Xtrans,index=rank_data.index, columns=rank_data.columns)

In [27]:
from skcriteria import Data, MIN, MAX
ranking = Data(
    rank_data_scaled,
    [MIN, MAX, MIN, MIN, MAX],
    cnames = rank_data.columns,
    weights = [1,10,1,1,1]
)

In [28]:
from skcriteria.madm import simple
dm = simple.WeightedSum(mnorm = "sum",wnorm = "sum")
dec = dm.decide(ranking)

In [29]:
rank_data['rank'] = dec.rank_
rank_data.sort_values(by=['rank'])

,price,number_of_comment,weight(kg),Negative,Positive,rank
163,6155000.0,0.0,42.0,0.151751,0.848249,1
170,10270000.0,1.0,57.2,0.009462,0.990538,2
243,14299000.0,0.0,65.6,0.199120,0.800880,3
139,7490000.0,1.0,40.0,0.041615,0.958385,4
143,10690000.0,1.0,63.0,0.041615,0.958385,5
185,4559000.0,1.0,35.2,0.041636,0.958364,6
166,4469000.0,1.0,43.0,0.055446,0.944554,7
218,8990000.0,1.0,47.0,0.055446,0.944554,8
241,5350000.0,0.0,38.4,0.238864,0.761136,9
97,14199000.0,1.0,72.0,0.072740,0.927260,10


In [30]:
cols_to_keep = ['index', 'name', 'brand']
group_data = group_data.drop(group_data.columns.difference(cols_to_keep), axis=1)

In [32]:
pd.concat([group_data, rank_data], axis=1, join="inner").to_csv('washing_machine_done.csv')